# Loading CSV into BanKAR database

This notebooks is used to load a CSV file generated by George BCR as data into the BanKAR database of tranzactions/transfers.

The CSV files should be placed in the `data` directory (needs to be created first) of this project.

In [8]:
// Replace with the path to your data file
val dataPath = "data/Statement_20240620_201005.csv"

## Initialization

Import stuff, connect to database and load the CSV.

In [9]:
import org.jetbrains.exposed.sql.*
import org.jetbrains.exposed.sql.transactions.*
import org.jetbrains.kotlinx.dataframe.*
import kotlinx.datetime.*
import kotlinx.datetime.format.*
import ro.bankar.database.*
import ro.bankar.banking.*

Database.connect("jdbc:postgresql://localhost:5432/bankar", user = "bankar", password = "bankar")
// Test connection
transaction {}

val rawData = DataFrame
    .readCSV(dataPath)
    .select("Transaction completion date", "Transaction completion hour", "Transaction's details",
        "Operation's reference", "Debit (amount)", "Credit (amount)")
    .dropNA()

rawData

Transaction completion date,Transaction completion hour,Transaction's details,Operation's reference,Debit (amount),Credit (amount)
01.04.2024,16:58,Tranzactie comerciant - Tranz: Nr car...,2024040107801303 Nota contabila 55823...,43.980000,0.000000
03.04.2024,13:53,Tranzactie efectuata prin George Bank...,2024040317491706 Ordin de plata 03....,0.000000,1100.000000
03.04.2024,17:57,"Google Pay, Tranzactie comerciant - T...",2024040318676235 Nota contabila 55901...,32.500000,0.000000
04.04.2024,18:35,"Google Pay, Tranzactie comerciant - T...",2024040423328808 Nota contabila 55904...,321.180000,0.000000
05.04.2024,18:20,Tranzactie comerciant - Tranz: Nr car...,2024040527647626 Nota contabila 55907...,100.000000,0.000000
05.04.2024,18:58,"Google Pay, Tranzactie comerciant - T...",2024040527873722 Nota contabila 55905...,37.000000,0.000000
06.04.2024,21:02,"Google Pay, Tranzactie comerciant - T...",2024040632446064 Nota contabila 55907...,12.000000,0.000000
08.04.2024,17:41,Tranzactie comerciant - Tranz: Nr car...,2024040839819380 Nota contabila 55913...,6.990000,0.000000
09.04.2024,19:27,Tranzactie comerciant - Tranz: Nr car...,2024040945971339 Nota contabila 55914...,19.300000,0.000000
10.04.2024,17:33,"Google Pay, Tranzactie comerciant - T...",2024041050448461 Nota contabila 55917...,17.000000,0.000000


## Data mapping

Convert the data from BCR's format to our format.

In [11]:
val dateFormat: DateTimeFormat<LocalDate> = LocalDate.Format {
    dayOfMonth(Padding.NONE)
    char('.')
    monthNumber()
    char('.')
    year()
}

enum class Type {
    Transfer, Tranzaction
}

val detailsPattern = Regex("""-Detalii: ([^.]*)\.?""")

val locationPattern = Regex("""Locatie: \w{8} \w{2} ([^.]+)\.""")

val detailsFormat = LocalDateTime.Format {
    dayOfMonth()
    char(' ')
    monthName(MonthNames.ENGLISH_ABBREVIATED)
    char(' ')
    year()
    chars(", ")
    hour()
    char(':')
    minute()
}

val details by column<String>()

val data = rawData
    .merge("Transaction completion date", "Transaction completion hour").by { dateFormat.parse(`Transaction completion date`).atTime(`Transaction completion hour`.toKotlinLocalTime()) }.into("timestamp")
    .merge("Credit (amount)", "Debit (amount)").by { `Credit (amount)` - `Debit (amount)` }.into("amount")
    .merge("Operation's reference").by { `Operation's reference`.slice(0..<16).toLong() }.into("reference")
    .rename("Transaction's details").into("details")
    .add("type") {
        if ("-Platitor" in details()) Type.Transfer else if ("Nr card" in details()) Type.Tranzaction else null
    }.dropNulls { "type"() }
    .add("title") { if ("type"<Type?>() == Type.Transfer) detailsPattern.find(details())!!.groupValues.get(1) else locationPattern.find(details())!!.groupValues.get(1) }
    .merge("details").by { "Payment on ${detailsFormat.format("timestamp"<LocalDateTime>())} at ${"title"<String>()}" }.into("details")

// Preview data
data

timestamp,details,reference,amount,type,title
2024-04-01T16:58,"Payment on 01 Apr 2024, 16:58 at Glov...",2024040107801303,-43.980000,Tranzaction,Glovo 31MAR BUQGJ2ZH5 BUCURESTI
2024-04-03T13:53,"Payment on 03 Apr 2024, 13:53 at Alim...",2024040317491706,1100.000000,Transfer,Alimentare cont
2024-04-03T17:57,"Payment on 03 Apr 2024, 17:57 at PREM...",2024040318676235,-32.500000,Tranzaction,PREMIER RESTAURANTS RO MANBRAGADIRU
2024-04-04T18:35,"Payment on 04 Apr 2024, 18:35 at ROMP...",2024040423328808,-321.180000,Tranzaction,ROMPETROL DWS R112 C2 BUCURESTI
2024-04-05T18:20,"Payment on 05 Apr 2024, 18:20 at Tran...",2024040527647626,-100.000000,Tranzaction,Trandafir Matei Dublin
2024-04-05T18:58,"Payment on 05 Apr 2024, 18:58 at PREM...",2024040527873722,-37.000000,Tranzaction,PREMIER RESTAURANTS RO MANBRAGADIRU
2024-04-06T21:02,"Payment on 06 Apr 2024, 21:02 at PREM...",2024040632446064,-12.000000,Tranzaction,PREMIER RESTAURANTS RO MANBRAGADIRU
2024-04-08T17:41,"Payment on 08 Apr 2024, 17:41 at GOOG...",2024040839819380,-6.990000,Tranzaction,GOOGLE *Google Play Ap g
2024-04-09T19:27,"Payment on 09 Apr 2024, 19:27 at PayU...",2024040945971339,-19.300000,Tranzaction,PayU*bolt
2024-04-10T17:33,"Payment on 10 Apr 2024, 17:33 at PREM...",2024041050448461,-17.000000,Tranzaction,PREMIER RESTAURANTS RO MANBRAGADIRU


## Configuration

Next, you need to specify with which BanKAR card to associate the tranzactions, and with which BanKAR account to associate the transfers.
You also need to set a second account that will be the recipient/sender of transfers made.

In [12]:
val cardId      = 6     // replace with ID of target card
val accountId   = 8     // replace with ID of own account
val otherAccIdd = 7     // replace with ID of account of another user

## Data loading

The data is loaded into the database.

### 1. Card transactions

In [13]:
transaction {
    val card = BankCard.findById(cardId)!!
    
    for (row in data.filter { type == Type.Tranzaction }) CardTransaction.new {
        reference = row.reference
        this.card = card
        amount = row.amount.absoluteValue.toBigDecimal()
        currency = Currency.ROMANIAN_LEU
        timestamp = row.timestamp.toInstant(TimeZone.currentSystemDefault())
        details = row.details
        title = row.title
    }
}

### 2. Bank transfers

In [14]:
transaction {
    val acc = BankAccount.findById(accountId)!!
    val otherAcc = BankAccount.findById(otherAccIdd)!!
    
    for (row in data.filter { type == Type.Transfer }) BankTransfer.new {
        val (sender, recipient) = if (row.amount >= 0) otherAcc to acc else acc to otherAcc
        this.sender = sender
        senderIban = sender.iban
        senderName = sender.user.fullName
        this.recipient = recipient
        recipientIban = recipient.iban
        recipientName = recipient.user.fullName
        amount = row.amount.absoluteValue.toBigDecimal()
        currency = Currency.ROMANIAN_LEU
        note = row.title
        timestamp = row.timestamp.toInstant(TimeZone.currentSystemDefault())
    }
}